# DA5401: Assignment 4
**GMM-Based Synthetic Sampling for Imbalanced Data**

**Course:** DA5401  
**Topic:** Credit Card Fraud (highly imbalanced)  

* **Name:** Harshit Shukla 
* **Roll No:** DA25S003


**Documentation of Code:**

In [2]:
# Importing necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report,
                            confusion_matrix, 
                            accuracy_score, 
                            roc_auc_score, 
                            roc_curve,
                            precision_recall_curve)
from sklearn.preprocessing import StandardScaler

from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline


import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.grid'] = True
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None) 


Random_State = 42   
np.random.seed(Random_State)